In [1]:
# -*- coding:utf-8 -*-
# !/usr/bin/python
# Author: Selvaria

# 模型构建和设计

import tensorflow as tf
import numpy as np
print(tf.__version__)

D:\Selvaria\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


2.0.0


In [2]:
#自定义MLP模型结构

class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()    # Flatten层将除第一维（batch_size）以外的维度展平
        self.dense1 = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)

    def call(self, inputs):         
        x = self.flatten(inputs)   
        x = self.dense1(x)    
        output = self.dense2(x)     
        return output
    
X = tf.random.uniform((2,20))
net = MLP()
net(X)

<tf.Tensor: id=62, shape=(2, 10), dtype=float32, numpy=
array([[ 0.14354907,  0.05338682, -0.40425608, -0.2587788 , -0.08983592,
         0.24334475,  0.08793818, -0.5242768 , -0.06720618,  0.17471312],
       [-0.01480664,  0.16989782, -0.4945268 , -0.17466378,  0.14202915,
        -0.02115927, -0.0175516 , -0.611707  , -0.027931  ,  0.10718849]],
      dtype=float32)>

In [3]:
# 正常的keras实现：也可以用add函数

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

model(X)

<tf.Tensor: id=117, shape=(2, 10), dtype=float32, numpy=
array([[-0.0521878 ,  0.0668958 , -0.06255163,  0.03062708, -0.14593492,
         0.10147388, -0.04072712, -0.11214042, -0.02359716, -0.13605078],
       [-0.2278136 , -0.04991286,  0.08795225,  0.17137462, -0.02623325,
         0.07074243, -0.05856334, -0.07146536,  0.12810664, -0.07756173]],
      dtype=float32)>

In [4]:
# 更灵活一些

class FancyMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.rand_weight = tf.constant(
            tf.random.uniform((20,20)))
        self.dense = tf.keras.layers.Dense(units=20, activation=tf.nn.relu)

    def call(self, inputs):         
        x = self.flatten(inputs)   
        x = tf.nn.relu(tf.matmul(x, self.rand_weight) + 1)
        x = self.dense(x)    
        while tf.norm(x) > 1:
            x /= 2
        if tf.norm(x) < 0.8:
            x *= 10
        return tf.reduce_sum(x)

In [5]:
# 定义后可直接通过add函数调用

class NestMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.net = tf.keras.Sequential()
        self.net.add(tf.keras.layers.Flatten())
        self.net.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
        self.net.add(tf.keras.layers.Dense(32, activation=tf.nn.relu))
        self.dense = tf.keras.layers.Dense(units=16, activation=tf.nn.relu)


    def call(self, inputs):         
        return self.dense(self.net(inputs))

net = tf.keras.Sequential()
net.add(NestMLP())
net.add(tf.keras.layers.Dense(20))
net.add(FancyMLP())

net(X)

<tf.Tensor: id=311, shape=(), dtype=float32, numpy=23.746525>